# Importing needed libraries

In [ ]:
import pandas as pd
import numpy as np 
import sqlite3

# Importing the dataset

In [ ]:
data = sqlite3.connect("/kaggle/input/sf-bay-area-bike-share/database.sqlite")

In [ ]:
pd.read_sql_query('''
    SELECT name, sql
    FROM sqlite_master
    WHERE type='table'
''', con=data)


In [ ]:
pd.read_sql_query('''
    SELECT *
    FROM station
''', con=data)


In [ ]:
pd.read_sql_query('''
    SELECT *
    FROM trip
''', con=data)


# Retrieve the ids of the bikes who have rented bikes for more than 10 days in total

In [ ]:
pd.read_sql_query('''
    SELECT bike_id
    FROM trip
    GROUP BY bike_id
    HAVING SUM(duration/(60*60*24)) > 10
''', con=data)


# Display the top 5 bikes that have been rented the most frequently, along with the count of their rentals

In [ ]:
pd.read_sql_query('''
    SELECT bike_id, COUNT(bike_id) AS rental_count
    FROM trip
    GROUP BY bike_id
    ORDER BY rental_count DESC
    LIMIT 5
''', con=data)

# Calculate the total revenue for each zip code area, considering the bike rental price per minute and the duration of the trip for all trips that exceed 30 minutes

In [ ]:
pd.read_sql_query('''
    SELECT zip_code, SUM((duration / 60) * 1) AS total_revenue
    FROM trip
    WHERE duration > 30 * 60
    GROUP BY zip_code;
''', con=data)


# Calculate the percentage of trips that were subscribed by customers (not casual users) for each season, considering only those trips that lasted longer than 15 minutes

In [ ]:
pd.read_sql_query('''
    SELECT 
        CASE 
            WHEN strftime('%m', start_date) IN ('12', '01', '02') THEN 'Winter'
            WHEN strftime('%m', start_date) IN ('03', '04', '05') THEN 'Spring'
            WHEN strftime('%m', start_date) IN ('06', '07', '08') THEN 'Summer'
            WHEN strftime('%m', start_date) IN ('09', '10', '11') THEN 'Fall'
        END AS season,
        COUNT(*) AS total_trips,
        SUM(CASE WHEN duration > 15 AND subscription_type = 'Subscriber' THEN 1 ELSE 0 END) AS subscribed_trips,
        (SUM(CASE WHEN duration > 15 AND subscription_type = 'Subscriber' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS percentage_subscribed
    FROM 
        trip
    GROUP BY 
        season;
''', con=data)


# Find the most popular station by calculating the ratio of the average number of trips per station to the average number of trips for all stations

In [ ]:
pd.read_sql_query('''
    SELECT start_station_name, AVG(trips_per_station) / AVG(total_trips) AS ratio
    FROM (
      SELECT start_station_name, COUNT(*)  AS trips_per_station
      FROM trip
      GROUP BY start_station_id) AS per_station,(
      SELECT COUNT(*) AS total_trips
      FROM trip
    ) AS total;
''', con=data)
